In [1]:
import json
import numpy as np
from scipy.spatial.distance import pdist
import random
from sklearn.model_selection import train_test_split




import math

# Load the config file from the configs folder
with open('../Configs/configGA.json') as f:
  config = json.loads(f.read())
import random
nodesbounds = config['config']['genetic_algorithm']["hiddenlayer_size"]
layerbounds = config['config']['genetic_algorithm']["hiddenlayer_number"]
learningrate = config['config']['genetic_algorithm']["learning_rates"]
batchsize = config['config']['genetic_algorithm']['batch_size']
mutationrate = config['config']['genetic_algorithm']['mutation_rate']
population_size = config['config']['genetic_algorithm']['population_size']


#create a list of possible nodes and layers
nodes = [i for i in range(nodesbounds[0],nodesbounds[1]+nodesbounds[2],nodesbounds[2])]
layers = [i for i in range(layerbounds[0],layerbounds[1]+layerbounds[2],layerbounds[2])]


## Global bounds on hyper parameters
nbound = len(nodes)
lbound =len(layers) 
ebound = len(learningrate) 
bbound = len(batchsize) 

#Determine the bitsize
nbitsize= len('{0:b}'.format(nbound-1))
lbitsize= len('{0:b}'.format(lbound-1))
ebitsize= len('{0:b}'.format(ebound-1))
bbitsize= len('{0:b}'.format(bbound-1))

bitsizes = [nbitsize,lbitsize,ebitsize,bbitsize]

upperbounds = [nbound,lbound,ebound,bbound]
lowerbounds = [0,0,0,0]


#Set the target for the genetic algorithm

targetsimple = [14,12,2,2]
lowesterror = 100
searchlog = {}

print(upperbounds)



def callCE(positionlist):
    """
    Determine if the selected position is already present in the library of positions or needs to be calculated
    """
    if str(positionlist) in searchlog:
        error = searchlog[str(positionlist)]
        print("library")
    else:
        error = calcError(targetsimple,positionlist)
        searchlog[str(positionlist)] = error
    return error

def calcError(target,positionlist):
    return (sum([abs(x - y) for x, y in zip(target, positionlist)]))
   

    
def checkBoundary(x,bound):
    """Checks a binary string against the boundary and sets it to the boundary in case it goes out of bounds"""
    if int(x,2) < bound:
        return x
    else:
        return '{0:b}'.format(bound-1)


def getOrder(population):
    """Return the order of the population list based on error rate"""
    populationscore = getScore(population)
    orderedpop = [pop for _, pop in sorted(zip(populationscore, population))]
    return orderedpop

def getScore(population):
    """ Get the error score for the current population"""
    print(f'Current population is: {population}')
    score = [callCE(pop) for pop in population]
    return score

def change(i):
    """Apply a chance of mutation to a an individual bit """
    if random.uniform(0, 1) < mutationrate:
        return i.replace('1', '2').replace('0', '1').replace('2', '0')
    else:
        return i

def mutate(i,bitstring):
    """Apply a chance of mutation to a bitstring"""
    output =""
    for c in bitstring:
        output+=change(c)
    
    return int(checkBoundary(output,i),2)
    
    
def mutateBatch(offspring1,offspring2):
    """Mutate all new offpsring"""
    return [mutate(upperbounds[i],x) for i,x in enumerate(offspring1)],[mutate(upperbounds[i],x) for i,x in enumerate(offspring2)]
    


def newGeneration(orderedpop):
    """Create a new generation"""
    parent1, parent2 = orderedpop[0],orderedpop[1]
    offspring1, offspring2 = mutateBatch(*getOffspring(parent1,parent2))
    
    return [parent1,parent2,orderedpop[2],orderedpop[3],offspring1,offspring2]


def getOffspring(parent1,parent2):
    """Create offpsring from the parents"""
    offspring1,offspring2 = ([],[])
    for index,(stat1,stat2) in enumerate(zip(parent1,parent2)):
        bit1 = bitsizes[index]
        bitparent1,bitparent2 = format(parent1[index],f'0{bit1}b'),format(parent2[index],f'0{bit1}b')
        cutoff = getCuttof(bitsizes[index])
        offspring1.append(bitparent1[:cutoff]+bitparent2[cutoff:])
        offspring2.append(bitparent2[:cutoff]+bitparent1[cutoff:])  
        print(f'these are bitwise: {offspring1} and {offspring2}')

        
    return offspring1,offspring2



def getCuttof(x):
    """Return the cuttof range"""
    return random.randrange(1,x)


def runmodel(population_size,iterations):
    """Run a genetic algorithm with a [population_size] size and [iterations] number of iterations"""
    population =[]
    for _ in range(population_size):
        population.append([random.randrange(nbound),random.randrange(lbound),random.randrange(ebound),random.randrange(bbound)])

    for i in range(iterations):
        orderedpop=getOrder(population)
        print(f'this is the best: {orderedpop[0]}')
        population=newGeneration(orderedpop)
        






[20, 20, 5, 5]


In [5]:
a = '{0:b}'.format(20)

print(a)
b = int(a,2)
print(b)

10100
20


In [3]:
#Test Boundary function

bound = 20
x ='{0:b}'.format(24) 

print(int(checkBoundary(x,bound),2))




19


In [6]:
# Test getscore function

targetsimple = [14,12,2,2]

population = []

for _ in range(population_size):
        population.append([random.randrange(nbound),random.randrange(lbound),random.randrange(ebound),random.randrange(bbound)])

print(population) 

print(getScore(population))

[[5, 16, 2, 4], [6, 19, 1, 1], [17, 9, 2, 3], [0, 15, 4, 4], [11, 1, 0, 2], [6, 9, 1, 3]]
Current population is: [[5, 16, 2, 4], [6, 19, 1, 1], [17, 9, 2, 3], [0, 15, 4, 4], [11, 1, 0, 2], [6, 9, 1, 3]]
[15, 17, 7, 21, 16, 13]


In [7]:
#test getorder

targetsimple = [14,12,2,2]

population = [[0, 12, 4, 2], [16, 5, 0, 0], [8, 9, 0, 4], [15, 2, 0, 4], [1, 5, 4, 1], [14, 19, 4, 3]]


print(getOrder(population)) 



Current population is: [[0, 12, 4, 2], [16, 5, 0, 0], [8, 9, 0, 4], [15, 2, 0, 4], [1, 5, 4, 1], [14, 19, 4, 3]]
[[14, 19, 4, 3], [8, 9, 0, 4], [16, 5, 0, 0], [15, 2, 0, 4], [0, 12, 4, 2], [1, 5, 4, 1]]


In [9]:
#test get offspring

parent1 = [16, 12, 4, 2] 
parent2 = [16, 5, 0, 0]


print(bitsizes)
def getOffspring(parent1,parent2):
    offspring1,offspring2 = ([],[])
    for index,(stat1,stat2) in enumerate(zip(parent1,parent2)):
        bit1 = bitsizes[index]
        bitparent1,bitparent2 = format(parent1[index],f'0{bit1}b'),format(parent2[index],f'0{bit1}b')
        print(bitparent1)
        print(bitparent2)
        cutoff = getCuttof(bitsizes[index])
        offspring1.append(bitparent1[:cutoff]+bitparent2[cutoff:])
        offspring2.append(bitparent2[:cutoff]+bitparent1[cutoff:])

    return offspring1,offspring2

getOffspring(parent1,parent2)



[5, 5, 3, 3]
10000
10000
01100
00101
100
000
010
000


(['10000', '00101', '100', '000'], ['10000', '01100', '000', '010'])

In [7]:



x = 3
def getCuttof(x):
    return random.randrange(1,x)

print(getCuttof(x))

1


In [10]:
## test full routine

runmodel(population_size,100)

Current population is: [[15, 9, 1, 1], [17, 11, 2, 3], [4, 8, 3, 0], [5, 19, 3, 3], [5, 19, 3, 2], [4, 1, 3, 4]]
this is the best: [17, 11, 2, 3]
10001
01111
01011
01001
010
001
011
001
Current population is: [[17, 11, 2, 3], [15, 9, 1, 1], [4, 8, 3, 0], [5, 19, 3, 2], [17, 9, 4, 3], [14, 10, 0, 4]]
library
library
library
library
this is the best: [17, 11, 2, 3]
10001
01110
01011
01010
010
000
011
100
Current population is: [[17, 11, 2, 3], [14, 10, 0, 4], [15, 9, 1, 1], [17, 9, 4, 3], [19, 10, 2, 2], [13, 11, 0, 1]]
library
library
library
library
this is the best: [13, 11, 0, 1]
01101
10001
01011
01011
000
010
001
011
Current population is: [[13, 11, 0, 1], [17, 11, 2, 3], [14, 10, 0, 4], [15, 9, 1, 1], [19, 14, 0, 3], [17, 19, 2, 1]]
library
library
library
library
this is the best: [13, 11, 0, 1]
01101
10001
01011
01011
000
010
001
011
Current population is: [[13, 11, 0, 1], [17, 11, 2, 3], [14, 10, 0, 4], [15, 9, 1, 1], [5, 13, 4, 3], [19, 11, 2, 1]]
library
library
library
libra

18
